In [1]:
import urllib.request
import os

In [2]:
url = "http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls"
filepath = "/Users/PChomeIM/pywork/Dataset/titanic3.xls"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url, filepath)
    print("downloaded:", result)

In [3]:
import numpy
import pandas as pd

In [4]:
all_df = pd.read_excel(filepath)

In [5]:
all_df[:2]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


In [6]:
all_df.shape

(1309, 14)

In [7]:
#挑選有意義的欄位
cols = ["survived", "name", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
all_df = all_df[cols]

In [8]:
all_df[:2]

,survived,name,pclass,sex,age,sibsp,parch,fare,embarked
0,1,"Allen, Miss. Elisabeth Walton",1,female,29.0000,0,0,211.3375,S
1,1,"Allison, Master. Hudson Trevor",1,male,0.9167,1,2,151.5500,S


In [9]:
#訓練時，暫時不需要name欄位
df = all_df.drop(['name'], axis=1)
df[:2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S


In [10]:
#查看所有欄位有幾筆資料含有null值
all_df.isnull().sum()

survived      0
name          0
pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

In [11]:
#將age欄位的null值填上平均值
age_mean = df['age'].mean()
df['age'] = df['age'].fillna(age_mean)

In [12]:
#將fare欄位的null值填上平均值
fare_mean = df['fare'].mean()
df['fare'] = df['fare'].fillna(fare_mean)

In [13]:
#轉換sex欄位為0和1
df['sex'] = df['sex'].map({'female':0, 'male':1}).astype(int)

In [14]:
#將embarked欄位轉成onehot表示
onehot_df = pd.get_dummies(data=df, columns=['embarked'])

In [15]:
onehot_df.isnull().sum()

survived      0
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked_C    0
embarked_Q    0
embarked_S    0
dtype: int64

In [16]:
onehot_df[:2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,0,0,1
1,1,1,1,0.9167,1,2,151.5500,0,0,1


In [17]:
#將df轉換成array
ndarray = onehot_df.values

In [18]:
ndarray.shape

(1309, 10)

In [19]:
ndarray[:2]

array([[   1.    ,    1.    ,    0.    ,   29.    ,    0.    ,    0.    ,
         211.3375,    0.    ,    0.    ,    1.    ],
       [   1.    ,    1.    ,    1.    ,    0.9167,    1.    ,    2.    ,
         151.55  ,    0.    ,    0.    ,    1.    ]])

In [20]:
#使用python slice擷取features和label資料
features = ndarray[:,1:] #取第1欄以後的欄位作為features
label = ndarray[:,0]     #取第0欄survived作為label

In [21]:
print(features.shape)
print(label.shape)

(1309, 9)
(1309,)


In [22]:
features[:2]

array([[   1.    ,    0.    ,   29.    ,    0.    ,    0.    ,  211.3375,
           0.    ,    0.    ,    1.    ],
       [   1.    ,    1.    ,    0.9167,    1.    ,    2.    ,  151.55  ,
           0.    ,    0.    ,    1.    ]])

In [24]:
#將特徵的值進行標準化，讓所有值介於0~1之間
from sklearn import preprocessing

In [25]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))
features = minmax_scale.fit_transform(features)

In [26]:
#查看標準化後的前2筆資料
features[:2]

array([[ 0.        ,  0.        ,  0.36116884,  0.        ,  0.        ,
         0.41250333,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.00939458,  0.125     ,  0.22222222,
         0.2958059 ,  0.        ,  0.        ,  1.        ]])